In [10]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

In [11]:
spark = (SparkSession.builder
        .appName("Ages")
        .getOrCreate())

22/12/12 02:58:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### Using RDD

In [12]:
#Agregate all ages by name and get the average name by age
#Create an RDD of tuples (name, age)

dataRDD = spark.sparkContext.parallelize([("Pedro", 38), ("Maria", 42), ("João", 12), ("Maria", 10)])

avgRDD = (dataRDD \
    .map(lambda x : (x[0], (x[1],1))) \
    .reduceByKey(lambda x,y : (x[0] + y[0], x[1] + y[1])) \
    .map(lambda x : (x[0], x[1][0]/x[1][1]))
    )

avgRDD.collect()

[('João', 12.0), ('Pedro', 38.0), ('Maria', 26.0)]

### Using Dartaframe

In [20]:
data_df = spark.createDataFrame([("Pedro", 38), ("Maria", 42), ("João", 12), ("Maria", 10)], ["nome", "idade"])

avg_df = data_df.groupBy("nome").agg(avg("idade"))

avg_df.show()

+-----+----------+
| nome|avg(idade)|
+-----+----------+
|Pedro|      38.0|
|Maria|      26.0|
| João|      12.0|
+-----+----------+



In [17]:
young = data_df.filter(data_df.idade < 18)
young.show()

+-----+-----+
| nome|idade|
+-----+-----+
| João|   12|
|Maria|   10|
+-----+-----+



In [18]:
old = data_df[data_df.idade > 18]
old.show()

+-----+-----+
| nome|idade|
+-----+-----+
|Pedro|   38|
|Maria|   42|
+-----+-----+



In [19]:
young.select(young.nome, young.idade + 1).show()

+-----+-----------+
| nome|(idade + 1)|
+-----+-----------+
| João|         13|
|Maria|         11|
+-----+-----------+



In [ ]:
spark.stop()